 ## 데이터는 Data.zip을 참고하시오.

In [1]:
cd C:\Users\USER\Quant

C:\Users\USER\Quant


In [2]:
import Quant_Strategy as Quant_Strategy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

In [3]:
path = '재무데이터.xlsx'
path_price = '수정주가.csv'
path_mkt = '시총데이터.csv'
path_endmonth = '결산월.csv'
path_kospiyn = '거래소데이터.csv'
path_delist_and_stop = '상장폐지데이터.csv'
### 추가된 데이터 ###
path_kospi_and_interest = 'kospi_interest_data.xlsx'
path_hedgeasset = '미국채데이터.xlsx'

In [4]:
cleaned_data = Quant_Strategy.preprocessing_path(path)
cleaned_price = Quant_Strategy.preprocessing_price(path_price)
cleaned_kospiyn = Quant_Strategy.preprocessing_kospiyn(path_kospiyn)
cleaned_mkt = Quant_Strategy.preprocessing_mktdata(path_mkt)
delist_data, stop_data = Quant_Strategy.preprocessing_stop_and_delist(path_delist_and_stop)
cleaned_endmonth = Quant_Strategy.preprocessing_period(path_endmonth)
cleaned_kospi_rf = Quant_Strategy.preprocessing_kospi_and_rf(path_kospi_and_interest)


 ### 문제 1. path_hedgeasset에는 헤지자산의 자산 가격이 존재한다.
 
 채권과 금 모두 달러가치이므로 이를 원화가치로 바꿔야한다. (채권, 골드, 현금의 인덱스 첫 날짜의 가치를 모두 10000으로 맞추시오.)
 
 preprocessing_hedgeasset(path_hedgeasset) 을 실행한 경우 다음과 같이 결과가 나온다. 
 
 | |mid_treasury|long_treasury|gold|cash|
|------|---|---|---|---|
| 1995-01-31|10000 |10000 |10000|10000|
| 1995-02-28|10322 |10264 |10031|10000|
| 1995-03-31|10207 |10148 |10254|10000|
| 1995-04-30|10846 |10927 |10068|10000|


In [5]:
def preprocessing_hedgeasset(path_hedgeasset) :
    hedge_asset = pd.read_excel(path_hedgeasset,index_col = 0)
    won_value_mid_treasury = hedge_asset['미국채10년'] * hedge_asset['달러환율']
    won_value_long_treasury = hedge_asset['미국장기채'] * hedge_asset['달러환율']
    won_value_gold = hedge_asset['골드'] * hedge_asset['달러환율']
    hedge_value = pd.concat([won_value_mid_treasury.rename('mid_treasury'),
                             won_value_long_treasury.rename('long_treasury'),
                             won_value_gold.rename('gold')], axis = 1)
    hedge_value = (hedge_value/hedge_value.iloc[0] * 10000).round(2)
    hedge_value['cash']= 10000
    return hedge_value

In [6]:
preprocessing_hedgeasset(path_hedgeasset)

mid_treasury  long_treasury      gold   cash
1995-01-31      10000.00       10000.00  10000.00  10000
1995-02-28      10322.15       10263.71  10030.94  10000
1995-03-31      10206.50       10147.95  10253.92  10000
1995-04-30      10231.50       10185.50  10067.58  10000
1995-05-31      10846.12       10927.28   9903.87  10000
...                  ...            ...       ...    ...
2020-09-30      67479.99      104989.78  70090.94  10000
2020-10-31      64274.22       98163.11  67343.92  10000
2020-11-30      62841.55       97118.80  62071.31  10000
2020-12-31      61760.72       94764.26  65439.17  10000
2021-01-31      62580.44       93695.38  64877.41  10000

[313 rows x 4 columns]

 ### 문제 2. 위에서 전처리한 헤지자산들을 사용하여 자산배분된 포트폴리오의 가치를 계산하시오. 
 
 기본 백테스트 변수 , weight_stock , weight_cash 를 더 추가하시오. 나머지 중기채, 장기채, 골드는 잔여비중을 동일가중한다. 
 
 채권과 골드는 리벨런싱할 때 가치가 (1-fee)만큼 감소합니다. 
 
 주식 포트폴리오는 이미 fee가 반영되어있습니다.
 
 기본적인 변수 셋팅은 다음과 같습니다.

In [17]:
##########################
## 함수에 들어갈 변수들 ##
##########################
start_day = '2000-01-31'
end_day = '2020-12-31'
Strategy = Quant_Strategy.F_score
rebalance_freq = 6
number_of_stock = 50
initial_money = 10000
fee = 0.007
w_stock = 0.3
w_cash = 0.1
cleaned_hedge_value = preprocessing_hedgeasset(path_hedgeasset)

In [14]:
################################
## 함수 본문에 들어가야할 내용 #
################################
PF  = Quant_Strategy.backtest(Strategy , start_day, end_day, cleaned_data, cleaned_price,
                              cleaned_mkt, delist_data, stop_data,cleaned_kospiyn,
                              rebalance_freq = rebalance_freq, number_of_stock = number_of_stock,
                              initial_money = initial_money, fee =fee)    
w_left = 1-w_stock - w_cash
w_each = np.round(w_left/3,4)
w = np.array([w_each,w_each,w_each,w_cash,w_stock]).reshape(1,-1)
rebalance_day_list = list(pd.date_range(start_day ,end_day, freq = 'M')[::rebalance_freq])
Port_DF = pd.DataFrame([])

all_assets = pd.concat([cleaned_hedge_value[PF.index[0]:PF.index[-1]],PF],axis = 1)
assets = all_assets/all_assets.iloc[0]    
    
adj_fee = np.array([1-fee, 1-fee, 1-fee, 1, 1])


#################################
## 여기서부터 직접 풀이하시오  ##
#################################

In [16]:
assets.head()

mid_treasury  long_treasury      gold  cash  Port_Value
2000-01-31      1.000000       1.000000  1.000000   1.0      1.0000
2000-02-29      1.033391       1.038542  1.045587   1.0      1.0862
2000-03-31      1.046721       1.048827  0.964914   1.0      1.1864
2000-04-30      1.038459       1.043740  0.960722   1.0      1.0437
2000-05-31      1.061216       1.060995  0.971048   1.0      0.8964

In [18]:
def asset_allocation_backtest(Strategy, start_day, end_day , cleaned_data, cleaned_price,
                              cleaned_mkt, delist_data, stop_data, cleaned_kospiyn, 
                              cleaned_hedge_value,w_stock = 0.3, w_cash = 0.1 , 
                              rebalance_freq = 3, number_of_stock = 50, initial_money = 10000, fee = 0.007) :
    
    ################################
    ## 함수 본문에 들어가야할 내용 #
    ################################
    
    #################################
    ## 여기서부터 직접 풀이하시오  ##
    #################################
    return Port_DF.reset_index().drop_duplicates(['index']).set_index('index')

In [19]:
def asset_allocation_backtest(Strategy, start_day, end_day , cleaned_data, cleaned_price,
                              cleaned_mkt, delist_data, stop_data, cleaned_kospiyn, 
                              cleaned_hedge_value,w_stock = 0.3, w_cash = 0.1 , 
                              rebalance_freq = 3, number_of_stock = 50, initial_money = 10000, fee = 0.008) :
    
    PF  = backtest(Strategy , start_day, end_day, cleaned_data, cleaned_price,
                                  cleaned_mkt, delist_data, stop_data,cleaned_kospiyn,
                                  rebalance_freq = rebalance_freq, number_of_stock = number_of_stock,
                                  initial_money = initial_money, fee =fee)    
    
    w_left = 1-w_stock - w_cash
    w_each = np.round(w_left/3,4)
    w = np.array([w_each,w_each,w_each,w_cash,w_stock]).reshape(1,-1)
    rebalance_day_list = list(pd.date_range(start_day ,end_day, freq = 'M')[::rebalance_freq])
    Port_DF = pd.DataFrame([])    
    
    all_assets = pd.concat([cleaned_hedge_value[PF.index[0]:PF.index[-1]],PF],axis = 1)
    assets = all_assets/all_assets.iloc[0]    
    
    adj_fee = np.array([1-fee, 1-fee, 1-fee, 1, 1])
    for i in range(len(rebalance_day_list)) : 
        start = rebalance_day_list[i]
        end = start + pd.DateOffset(months = rebalance_freq, day = 31) 
        P = assets[start:end]
        Port_Value = initial_money * w * P/P.iloc[0]  * adj_fee
        Port_DF = pd.concat([Port_DF, Port_Value.sum(1)],axis = 0)
        initial_money = Port_Value.sum(1).iloc[-1] 
    Port_DF.columns = ['Port_Value']
    return Port_DF.reset_index().drop_duplicates(['index']).set_index('index')

 ### 문제 3. 지금까지 만든 함수들을 .py에 추가하시오.